In [1]:
import torch
import sys
sys.path.insert(0, '/xdisk/msurdeanu/fanluo/hotpotQA/')
from util import get_buckets

import numpy as np
import os
import time
import json

%matplotlib inline
import random
import pandas as pd
from matplotlib import pyplot as plt


In [2]:
word_emb_file = "/xdisk/msurdeanu/fanluo/hotpotQA/word_emb.json"
with open(word_emb_file, "r") as fh:
    word_mat = np.array(json.load(fh), dtype=np.float32)
with open('/xdisk/msurdeanu/fanluo/hotpotQA/idx2word.json', 'r') as fh:	
    idx2word_dict = json.load(fh)

In [3]:
train_buckets = get_buckets('/xdisk/msurdeanu/fanluo/hotpotQA/train_record.pkl')
X_train = train_buckets[0]

In [ ]:
question_data = {}    
question_avg_embedding = np.array([])
question_list = []

for i in range(len(X_train)):
    ques_idxs = X_train[i]['ques_idxs']
    ques_idxs = ques_idxs[ques_idxs > 1]  # 0 is padding, 1 is unknown, questions longer than ques_limit already been discarded in prepro.py
    ques_words = ' '.join([idx2word_dict[str(int(idx))] for idx in ques_idxs])	
    question_list.append(ques_words)
    question_word_embedding_mat = word_mat[ques_idxs]
    question_embedding = np.mean(question_word_embedding_mat, 0)  # average of word embedding as question embedding

    if i == 0:
        question_avg_embedding = question_embedding 
    else:
        question_avg_embedding = np.vstack((question_avg_embedding, question_embedding))

    
question_data['text'] = question_list
question_data['emb'] = question_avg_embedding

In [ ]:
# ref to https://www.kaggle.com/colinmorris/visualizing-embeddings-with-t-sne
from sklearn.manifold import TSNE
tsne = TSNE(random_state=1, n_iter=15000, metric="cosine")

embs = tsne.fit_transform(question_data['emb'])
# Add to dataframe for convenience
df['x'] = embs[:, 0]
df['y'] = embs[:, 1]